<a href="https://colab.research.google.com/github/ParvinGhaffarzadeh/GA_NN_2_2021/blob/main/GA_NN_2_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from random import randint
import random
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error as mae

In [2]:
import pandas as pd

In [3]:
#Load Data
from google.colab import files
uploaded= files.upload()

Saving myoutput1.xls to myoutput1.xls


In [4]:
df = pd.read_excel('myoutput1.xls')
#df = pd.read_excel('C:/Users/asus/Desktop/samane/Data2SaraO.xls')

df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,cluster
0,400000000,2880000,0,2,1,0,2,1,2,1,26,8,1
1,500000000,3400000,0,2,1,0,1,1,2,1,16,8,1
2,80000000,976800,0,2,1,0,2,4,1,1,12,8,0
3,720000000,11520009,0,2,1,0,2,1,3,1,23,8,1
4,25000000,300000,0,2,1,0,3,4,1,1,16,7,0


In [5]:
X = df.to_numpy()

In [6]:
np.random.seed = 0
X = X[np.random.permutation(X.shape[0])]

y = X[:, -1]
X = X[:, :-1]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:


def inicializacao_populacao_mlp(size_mlp):
    activation = ['identity','logistic', 'tanh', 'relu']
    solver = ['lbfgs','sgd', 'adam']
    pop =  np.array([[random.choice(activation), random.choice(solver), randint(2,100),randint(2,50)]])
    for i in range(0, size_mlp-1):
        pop = np.append(pop, [[random.choice(activation), random.choice(solver), randint(2,50),randint(2,50)]], axis=0)
    return pop

def cruzamento_mlp(pai_1, pai_2):
    child = [pai_1[0], pai_2[1], pai_1[2], pai_2[3]]    
    return child

def mutacao_mlp(child, prob_mut):
    child_ = np.copy(child)
    for c in range(0, len(child_)):
        if np.random.rand() >= prob_mut:
            k = randint(2,3)
            child_[c,k] = int(child_[c,k]) + randint(1, 4)
    return child_


def function_fitness_mlp(pop, X_train, y_train, X_test, y_test): 
    fitness = []
    j = 0
    for w in pop:
        clf = MLPClassifier(learning_rate_init=0.09, activation=w[0], solver = w[1], alpha=1e-5, hidden_layer_sizes=(int(w[2]), int(w[3])),  max_iter=1000, n_iter_no_change=80)

        try:
            clf.fit(X_train, y_train)
            f = accuracy_score(clf.predict(X_test), y_test)
            

            fitness.append([f, clf, w])
        except:
            pass
    return fitness#


def ag_mlp(X_train, y_train, X_test, y_test, num_epochs = 10, size_mlp=10, prob_mut=0.8):
    pop = inicializacao_populacao_mlp(size_mlp)
    fitness = function_fitness_mlp(pop,  X_train, y_train, X_test, y_test)
    pop_fitness_sort = np.array(list(reversed(sorted(fitness,key=lambda x: x[0]))))

    for j in range(0, num_epochs):
        length = len(pop_fitness_sort)
        #seleciona os pais
        parent_1 = pop_fitness_sort[:,2][:length//2]
        parent_2 = pop_fitness_sort[:,2][length//2:]

        #cruzamento
        child_1 = [cruzamento_mlp(parent_1[i], parent_2[i]) for i in range(0, np.min([len(parent_2), len(parent_1)]))]
        child_2 = [cruzamento_mlp(parent_2[i], parent_1[i]) for i in range(0, np.min([len(parent_2), len(parent_1)]))]
        child_2 = mutacao_mlp(child_2, prob_mut)
        
        #calcula o fitness dos filhos para escolher quem vai passar pra próxima geração
        fitness_child_1 = function_fitness_mlp(child_1,X_train, y_train, X_test, y_test)
        fitness_child_2 = function_fitness_mlp(child_2, X_train, y_train, X_test, y_test)
        pop_fitness_sort = np.concatenate((pop_fitness_sort, fitness_child_1, fitness_child_2))
        sort = np.array(list(reversed(sorted(pop_fitness_sort,key=lambda x: x[0]))))
        
        #seleciona individuos da proxima geração
        pop_fitness_sort = sort[0:size_mlp, :]
        best_individual = sort[0][1]
        
    return best_individual

melhor_result = ag_mlp(X_train, y_train, X_test, y_test, num_epochs = 10, size_mlp=20, prob_mut=0.9)
print (accuracy_score(melhor_result.predict(X_test), y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_base.py:66: RuntimeWarning: invalid value encountered in subtract
  tmp = X - X.max(axis=1)[:, np.newaxis]
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: Visibl